1. Write a function that converts number representation (bin<->dec<->hex)

In [ ]:
#Exercise 1
def convert(fromnumber,tonumber,number):
    if fromnumber=="binary" and tonumber=="decimal":
        return int(number,2)
    if fromnumber=="binary" and tonumber=="hexadecimal":
        return hex(int(number))
    if fromnumber=="decimal" and tonumber=="hexadecimal":
        return hex(int(number))
    if fromnumber=="decimal" and tonumber=="binary":
        return bin(int(number))
    if fromnumber=="hexadecimal" and tonumber=="decimal":
        return int(number,16)
    if fromnumber=="hexadecimal" and tonumber=="binary":
        return bin(hex(number))
fromnumber=input("which type would you convert from?")
tonumber=input("which type would you convert to?")
number=input("which number would you convert?")
print(convert(fromnumber,tonumber,number))

# --------- there is always easy way in python...

def f(a):
    a_bin=bin(a)
    a_hex=hex(a)
    a_dec=a

    print("Binary:",a_bin)
    print("Hexadecimal:",a_hex)
    print("Decimal:",a)

2. Write a function that converts a 32 bit word into a single precision floating point (i.e. interprets the various bits as sign, mantissa and exponent)

3. Write a program to determine the underflow and overflow limits (within a factor of 2) for python on your computer.

In [ ]:
#Exercise 3 

N=2000
underflow=1
overflow=1
factor=2

for i in range(N):
    underflow=underflow/2
    overflow=overflow*2
    print("|%2d"%i,"|","\t\t","|%2.5e"%underflow,"|","\t\t","|%2.5e"%overflow,"|")

# ----------------
i=0
a=1
while a!=0:
    a=a/2
    i += 1
print pow(2,-i+1)    

4. Write a program to determine the machine precision

In [8]:
x=1
def precision(k):
    b=1
    i=0
    while k != k+b:
        k=k+b
        b=b/2
        i += 1
    print (pow(2, -i+1))         
precision(x)

2.220446049250313e-16


5. Write a function that takes in input three parameters a, b and c and prints out the two solutions to the quadratic equation using the standard formula: 

In [ ]:
import math
def roots(a,b,c):
    x1=((-b+(math.sqrt((b**2)-(4*a*c))))*(-b+(math.sqrt((b**2)-(4*a*c)))))/((2*a)*(-b+(math.sqrt((b**2)-(4*a*c)))))
    x2=((-b-(math.sqrt((b**2)-(4*a*c))))*(-b-(math.sqrt((b**2)-(4*a*c)))))/((2*a)*(-b-(math.sqrt((b**2)-(4*a*c)))))
    return x1,x2
print(roots(0.001,1000,0.001))

#------------------

x=0
def roots2(a,b,c):
    x=(-b+pow(b*b-4*a*c,0.5))/2*a
    print("1. root: ",x)
    x=(-b-pow(b*b-4*a*c,0.5))/2*a
    print("2. root: ",x)
print(roots2(0.001,1000,0.001))    

6. Write a program that implements the function 

In [ ]:
def f(x):
    return x
def derivative(x):
    h=1./10.
    slope=(f(x+h)-f(x))/h
    return slope

print(derivative(1))


7. a  Write a programe to compute the integral with $N=100$. How does the result compares to the true value?

In [ ]:
import math
def f(x):
    return math.sqrt(1-x**2)
def integral(starting,ending,N):
    width=(float(ending)-float(starting))/N
    Sum=0
    for i in range(N):
        height=f(starting+i*width)
        area=height*width
        Sum+=area
    return Sum
print(integral(-1,1,100))

7. b : How much can N be increased if the computation needs to be run in less than a second? What is the gain in running it for 1 minute?

In [ ]:
import math
import time
N=3000000
def f(x):
    return math.sqrt(1-x**2)
def integral(starting,ending,N):
    stattime=0
    endtime=0
    starttime=time.time()
    width=(float(ending)-float(starting))/N
    Sum=0
    for i in range(N):
        height=f(starting+i*width)
        area=height*width
        Sum+=area
    endtime=time.time()
    return endtime-starttime
while(integral(-1,1,N)>=1):
    print(N,integral(-1,1,N))
    N-=100000
print("The minimum N in less than 1 munite is:",N,"Time is:",integral(-1,1,N))